In [37]:
import boto3
import pandas as pd
import os
import gzip
import numpy as np

keys = pd.read_csv('rootkey.csv')
aws_access_key_id = keys['Access key ID'][0]
secret_access = keys['Secret access key'][0]

os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
os.environ['AWS_SECRET_ACCESS_KEY'] = secret_access

if not os.path.exists('data'):
    os.mkdir('data')

s3 = boto3.client('s3')
bucket_name = 'tripdata'
prefix = '2021'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

In [17]:
import zipfile

# Function to download and extract zip files
def download_and_extract_zip(s3_client, bucket, object_key, local_zip_file):
    # Download the zip file
    s3_client.download_file(bucket, object_key, local_zip_file)
    print(f"Downloaded {object_key}")

    # Extract the zip file
    with zipfile.ZipFile(local_zip_file, 'r') as zip_ref:
        zip_ref.extractall('./data/')
        print(f"Extracted {local_zip_file}")

# Initialize a boto3 S3 client
s3 = boto3.client('s3')


# Define the bucket name
bucket_name = 'tripdata'

# Define the prefix for 2021 data
prefix = '2021'

# List the objects in the bucket with the specified prefix
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# Check if the bucket has contents with the specified prefix
if 'Contents' in response:
    # Iterate through the objects with the prefix of 2021
    for item in response['Contents']:
        file_name = item['Key']
        # Check if the file name ends with '.zip'
        if file_name.endswith('.zip'):
            # Define a local path to save the zip file
            local_zip_file = file_name.split('/')[-1]
            # Download and extract the zip file
            download_and_extract_zip(s3, bucket_name, file_name, 'data/' + local_zip_file)
        else:
            print(f"Skipping {file_name}, not a ZIP file")
else:
    print("No ZIP files found for the year 2021.")

Downloaded 202101-citibike-tripdata.csv.zip
Extracted data/202101-citibike-tripdata.csv.zip
Downloaded 202102-citibike-tripdata.csv.zip
Extracted data/202102-citibike-tripdata.csv.zip
Downloaded 202103-citibike-tripdata.csv.zip
Extracted data/202103-citibike-tripdata.csv.zip
Downloaded 202104-citibike-tripdata.csv.zip
Extracted data/202104-citibike-tripdata.csv.zip
Downloaded 202105-citibike-tripdata.csv.zip
Extracted data/202105-citibike-tripdata.csv.zip
Downloaded 202106-citibike-tripdata.csv.zip
Extracted data/202106-citibike-tripdata.csv.zip
Downloaded 202107-citibike-tripdata.csv.zip
Extracted data/202107-citibike-tripdata.csv.zip
Downloaded 202108-citibike-tripdata.csv.zip
Extracted data/202108-citibike-tripdata.csv.zip
Downloaded 202109-citibike-tripdata.csv.zip
Extracted data/202109-citibike-tripdata.csv.zip
Downloaded 202110-citibike-tripdata.csv.zip
Extracted data/202110-citibike-tripdata.csv.zip
Downloaded 202111-citibike-tripdata.csv.zip
Extracted data/202111-citibike-tripd

In [38]:
datafiles = sorted(list(os.listdir('data')))

def adjust_colnames(df: pd.DataFrame) -> pd.DataFrame:
    if 'tripduration' in df.columns:
        df = df.rename(
            columns={
                'starttime': 'started_at',
                'stoptime': 'ended_at',
                'start station id': 'start_station_id',
                'start station name': 'start_station_name',
                'start station latitude': 'start_lat',
                'start station longitude': 'start_lng',
                'end station id': 'end_station_id',
                'end station name': 'end_station_name',
                'end station latitude': 'end_lat',
                'end station longitude': 'end_lng',
                'usertype': 'member_casual',
            })
        df = df.drop(columns=['tripduration', 'birth year', 'bikeid', 'gender'])
        df['rideable_type'] = np.nan
        df['ride_id'] = np.nan
    return df

for file in datafiles:
    if file.endswith('.csv'):
        data = adjust_colnames(pd.read_csv('data/' + file, low_memory=False))
        print(file)
        print(len(data.columns))

202101-citibike-tripdata.csv
12
202102-citibike-tripdata.csv
13
202103-citibike-tripdata.csv
13


KeyboardInterrupt: 

In [35]:
data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,72D1C9F1A2FEB359,classic_bike,2021-12-04 17:53:23,2021-12-04 18:05:37,Irving Ave & Halsey St,4695.04,Central Ave & Himrod St,4713.01,40.694670,-73.906630,40.696760,-73.923020,casual
1,8BE7469E1A90508C,electric_bike,2021-12-04 17:35:04,2021-12-04 17:52:55,W 25 St & 6 Ave,6215.04,W 29 St & 9 Ave,6416.06,40.743954,-73.991449,40.750073,-73.998393,casual
2,AF9EDCC2A347E607,electric_bike,2021-12-13 23:59:41,2021-12-14 00:08:31,Irving Ave & Halsey St,4695.04,Harman St & Seneca Ave,5002.09,40.694670,-73.906630,40.705770,-73.912920,casual
3,495CF3C8A1B7FED8,classic_bike,2021-12-22 12:22:38,2021-12-22 12:25:46,Lenox Ave & W 115 St,7627.10,Central Park North & Adam Clayton Powell Blvd,7617.07,40.801194,-73.950074,40.799484,-73.955613,member
4,1C138E3D55FCC7C1,classic_bike,2021-12-16 10:15:22,2021-12-16 10:32:02,W 26 St & 10 Ave,6382.05,W Broadway & Spring St,5569.06,40.749718,-74.002950,40.724947,-74.001659,member
